# Soundfile Library initial test

In [5]:
import soundfile as sf
import os
import time

def stream_wav_file(file_path, block_size=1024):
    """
    Chunk audio loading for .wav files
    """
    with sf.SoundFile(file_path, 'r') as f:
        while True:
            chunk = f.read(block_size)
            if not len(chunk):
                break
            yield chunk

path = "/home/javastral/GIT/ANE2-GCPDS/Datasets/ANEaudios/"
wav_files = [file for file in os.listdir(path) if file.endswith(".wav")]
wav_files.sort()

audio_vectors = []
total_files = len(wav_files)
file_stats = []

print("--- Processing Audio Files ---")

total_time = 0
total_size_mb = 0

for file in wav_files:
    file_path = os.path.join(path, file)
    file_size_bytes = os.path.getsize(file_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    total_size_mb += file_size_mb

    start_time = time.time()
    for chunk in stream_wav_file(file_path):
        audio_vectors.append(chunk)
    end_time = time.time()
    loading_time = end_time - start_time
    total_time += loading_time

    file_stats.append({
        "file_name": file,
        "size_mb": file_size_mb,
        "loading_time_sec": loading_time
    })

# --- Summary of Results ---
print("\n--- Audio File Statistics Summary ---")
print(f"Total number of files processed: {total_files}\n")

if file_stats:
    # Print table header
    print(f"{'File Name':<30} | {'Size (MB)':>10} | {'Loading Time (s)':>15}")
    print("-" * 60)

    # Print data for each file
    for stats in file_stats:
        print(f"{stats['file_name']:<30} | {stats['size_mb']:>10.2f} | {stats['loading_time_sec']:>15.4f}")
else:
    print("No .wav files found in the specified directory.")


print("---------Total stats-------")
print(f"total time: {total_time}, Total size MB: {total_size_mb}")

--- Processing Audio Files ---

--- Audio File Statistics Summary ---
Total number of files processed: 1205

File Name                      |  Size (MB) | Loading Time (s)
------------------------------------------------------------
100.0_0_.wav                   |       0.32 |          0.0026
100.0_1_.wav                   |       0.32 |          0.0020
100.0_2_.wav                   |       0.32 |          0.0019
100.0_3_.wav                   |       0.32 |          0.0018
100.0_4_.wav                   |       0.32 |          0.0039
100.0_5_.wav                   |       0.32 |          0.0042
100.1_0_.wav                   |       0.32 |          0.0038
100.1_1_.wav                   |       0.32 |          0.0020
100.1_2_.wav                   |       0.32 |          0.0035
100.1_3_.wav                   |       0.32 |          0.0045
100.1_4_.wav                   |       0.32 |          0.0044
100.1_5_.wav                   |       0.32 |          0.0035
100.2_0_.wav           

# Upping chunksize with soundfile

In [11]:
# -*- coding: utf-8 -*-
"""
Analyzes audio files in a directory to measure I/O performance.

This script provides utilities to stream and analyze WAV audio files from a
specified directory. It calculates and displays statistics about the time taken
to load each file and provides a summary of the total processing time and file
sizes. The primary focus is on measuring I/O performance by reading files in chunks.

Example:
    To run the analysis on a directory containing .wav files:

        $ python your_script_name.py

    The script will then process all .wav files in the `/content/` directory
    and print a summary of the analysis.
"""

import os
import time
from typing import Iterator, Dict, Any, List
import numpy as np
import soundfile as sf

def stream_wav_file(file_path: str, block_size: int = 65536) -> Iterator[np.ndarray]:
    """Lazily loads a .wav file in chunks using a generator.

    This function reads a WAV file piece by piece, yielding chunks of audio
    data as NumPy arrays. This approach is memory-efficient for large files
    as it doesn't load the entire file into memory at once.

    Args:
        file_path (str): The path to the WAV file.
        block_size (int): The number of audio frames to read per chunk.

    Yields:
        np.ndarray: A chunk of the audio file.

    Raises:
        FileNotFoundError: If the specified file_path does not exist.
        sf.SoundFileError: If the file is not a valid WAV file or is corrupted.
    """
    try:
        with sf.SoundFile(file_path, 'r') as f:
            while f.tell() < f.frames:
                chunk = f.read(block_size)
                if chunk.size == 0:
                    break
                yield chunk
    except (FileNotFoundError, sf.SoundFileError) as e:
        print(f"Error processing file {file_path}: {e}")
        raise

def analyze_audio_files(directory: str) -> None:
    """Analyzes all .wav files in a directory, calculating loading times.

    This function scans a directory for WAV files, measures the time it takes
    to read each file using a streaming approach, and then prints a formatted
    summary of the results, including individual file statistics and overall totals.

    Args:
        directory (str): The directory containing the .wav files.
    """
    try:
        wav_files = sorted([
            file for file in os.listdir(directory)
            if file.lower().endswith(".wav")
        ])
    except FileNotFoundError:
        print(f"Error: The directory '{directory}' was not found.")
        return

    if not wav_files:
        print("No .wav files found in the specified directory.")
        return

    file_stats: List[Dict[str, Any]] = []
    total_time = 0.0
    total_size_mb = 0.0
    total_files = len(wav_files)

    print("--- Processing Audio Files ---")

    for file_name in wav_files:
        file_path = os.path.join(directory, file_name)
        try:
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = file_size_bytes / (1024 * 1024)
            total_size_mb += file_size_mb

            start_time = time.time()
            # The loop consumes the generator, triggering the file reading.
            for _ in stream_wav_file(file_path):
                # We are only measuring I/O time, so no processing is needed here.
                pass
            end_time = time.time()

            loading_time = end_time - start_time
            total_time += loading_time

            file_stats.append({
                "file_name": file_name,
                "size_mb": file_size_mb,
                "loading_time_sec": loading_time
            })
        except Exception as e:
            print(f"Could not process {file_name}. Reason: {e}")

    # --- Summary of Results ---
    print("\n--- Audio File Statistics Summary ---")
    print(f"Total number of files processed: {total_files}\n")

    # Print table header.
    print(f"{'File Name':<30} | {'Size (MB)':>10} | {'Loading Time (s)':>15}")
    print("-" * 60)

    # Print data for each file.
    for stats in file_stats:
        print(f"{stats['file_name']:<30} | {stats['size_mb']:>10.2f} | {stats['loading_time_sec']:>15.4f}")

    print("-" * 60)
    print(f"{'TOTAL':<30} | {total_size_mb:>10.2f} | {total_time:>15.4f}")

if __name__ == "__main__":
    # The directory containing the audio files to be analyzed.
    path_to_audio = "/home/javastral/GIT/ANE2-GCPDS/Datasets/ANEaudios/"
    analyze_audio_files(path_to_audio)

--- Processing Audio Files ---

--- Audio File Statistics Summary ---
Total number of files processed: 1205

File Name                      |  Size (MB) | Loading Time (s)
------------------------------------------------------------
100.0_0_.wav                   |       0.32 |          0.0018
100.0_1_.wav                   |       0.32 |          0.0008
100.0_2_.wav                   |       0.32 |          0.0007
100.0_3_.wav                   |       0.32 |          0.0007
100.0_4_.wav                   |       0.32 |          0.0008
100.0_5_.wav                   |       0.32 |          0.0008
100.1_0_.wav                   |       0.32 |          0.0011
100.1_1_.wav                   |       0.32 |          0.0011
100.1_2_.wav                   |       0.32 |          0.0011
100.1_3_.wav                   |       0.32 |          0.0012
100.1_4_.wav                   |       0.32 |          0.0010
100.1_5_.wav                   |       0.32 |          0.0009
100.2_0_.wav           

# Pseudocode: Stream and Analyze WAV Audio Files

## **Function** `StreamWavFile`
*Analyzes all WAV files in a directory to measure I/O performance.*

-   **Input:** `filePath` (string), `blockSize` (integer)
-   **Yields:** `chunk` (an array of audio data)

**`try`**:
      Open the audio file at `filePath` and assign it to `f`.
     **`while`** there are more frames to read in `f` **`do`**:
          `chunk` ← read `blockSize` frames from `f`.
          **`if`** `chunk` is empty **`then`**:
              `break` from the loop.
          **`end if`**
          `yield` the `chunk`.
      **`end while`**
 **`catch`** `Exception` as `e`:
     Print an error message for `filePath`.
     `raise` the exception `e`.
 **`end try`**

---

## **Function** `AnalyzeAudioFiles`
*Analyzes all WAV files in a directory to measure I/O performance.*

-   **Input:** `directory` (string)

 **`try`**:
      `wavFiles` ← Get a sorted list of all files ending with `.wav` in the `directory`.
  **`catch`** `FileNotFoundError`:
      Print an error that the `directory` was not found.
      `return`.
  **`end try`**
  
  **`if`** `wavFiles` is empty **`then`**:
      Print a message that no `.wav` files were found.
     `return`.
 **`end if`**

 Initialize `fileStats` ← `[]` (an empty list).
 Initialize `totalTime` ← `0.0`.
 Initialize `totalSizeMB` ← `0.0`.
 `totalFiles` ← the number of items in `wavFiles`.

 Print "--- Processing Audio Files ---".

 **`for`** each `fileName` in `wavFiles` **`do`**:
     `filePath` ← join `directory` and `fileName`.
     **`try`**:
         `fileSizeBytes` ← get the size of `filePath` in bytes.
         `fileSizeMB` ← calculate the size in megabytes:
            $$ \text{fileSizeMB} = \frac{\text{fileSizeBytes}}{1024 \times 1024} $$
         `totalSizeMB` ← `totalSizeMB` + `fileSizeMB`.

         `startTime` ← current system time.
         **`for`** each `chunk` yielded by `StreamWavFile(filePath)` **`do`**:
             > *This loop consumes the generator to trigger file I/O.*
             `pass`.
         **`end for`**
         `endTime` ← current system time.

         `loadingTime` ← `endTime` - `startTime`.
         `totalTime` ← `totalTime` + `loadingTime`.

         Append a new dictionary `{ "file_name": fileName, "size_mb": fileSizeMB, "loading_time_sec": loadingTime }` to `fileStats`.
     **`catch`** `Exception` as `e`:
         Print an error message for the current `fileName`.
     **`end try`**
 **`end for`**

 Print a summary title: "--- Audio File Statistics Summary ---".
 Print the total number of files processed.
 Print table headers: "File Name", "Size (MB)", "Loading Time (s)".

 **`for`** each `stats` record in `fileStats` **`do`**:
     Print a formatted row with the file's name, size, and loading time.
 **`end for`**

 Print a table footer with the total size and total time.

---

## **Main Execution Block**
  `path_to_audio` ← `"/content/"`.
  Call `AnalyzeAudioFiles` with `path_to_audio`.```